# Keras Remote Training
Uses a dataset of wine attributes to demonstrate a template for remote training with Keras. The talos module is used for hyperparameter optimization.

## Control

In [ ]:
control = {
    'pip': False,                         # install packages in this notebook
    'experiment_name': 'wine_quality_1',  # for talos scan
    'run_scan': False,                    # whether to execute scan
    'upload_results': False,              # upload results to S3 or use local file
    'download_results': True,             # download results from S3 or use local file
    's3bucket': 'keras-remote-training'   # can be None if using only local files
}

In [ ]:
results_filename = control['experiment_name'] + '.csv'
deploy_filename = control['experiment_name'] + '.zip'

## Environment setup

In [ ]:
if control['pip'] == False:
    print('Skipping install of python packages')

else:
    !pip install -r requirements.txt

In [ ]:
import os
import pwd
from collections import OrderedDict

import boto3
import botocore

import numpy as np
import pandas as pd
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

import matplotlib
if pwd.getpwuid(os.getuid())[0] == 'kerasdeploy':
    print('Using agg backend for matplotlib')
    matplotlib.use("agg")
import matplotlib.pyplot as plt


import seaborn as sns

import tensorflow as tf
import keras

import talos as ta
from talos import model as ta_model
from talos.metrics.keras_metrics import precision, recall, f1score, matthews

In [ ]:
print('Tensorflow version: %s' % tf.VERSION)
print('Keras version: %s' % keras.__version__)
print('Talos version: %s' % ta.__version__)

## Data Preparation

Citation for data source:

P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis. Modeling wine preferences by data mining from physicochemical properties. In Decision Support Systems, Elsevier, 47(4):547-553, 2009.

In [ ]:
df = pd.read_csv('winequality-red.csv')

In [ ]:
df.head(5)

In [ ]:
df['quality'].hist()

Create normalized X numpy array

In [ ]:
X = df.iloc[:,0:-1].values

In [ ]:
X_prenorm_min = X.min(0)
X_prenorm_ptp = X.ptp(0)
X = (X - X_prenorm_min) / X_prenorm_ptp

Create Y numpy array with shape (observations, classes=2)

In [ ]:
Y = np.zeros((X.shape[0],2), dtype=np.float32)

In [ ]:
threshold = 7
Y[:,0] = (df['quality'] < threshold)*1.0
Y[:,1] = (df['quality'] >= threshold)*1.0

View balance of classes and calculate weights

In [ ]:
Y_frac_lowq = np.sum(Y[:,0]) / Y.shape[0]
Y_frac_highq = np.sum(Y[:,1]) / Y.shape[0]
print('Fraction data of class low/high quality:\n%f, %f' % (Y_frac_lowq, Y_frac_highq))

In [ ]:
Y_array = np.argmax(Y, axis=1)
class_weights = compute_class_weight('balanced', np.unique(Y_array), Y_array)
class_weights = dict(enumerate(class_weights))
print('Class weights:\n%s' % class_weights)

## Run model

Create model function compatible with talos

In [ ]:
def get_model(x_train, y_train, x_val, y_val, params):
    """
    Compiles and fits model using params as defined by talos.
    Returns model_fit (history) object and model object.
    """
        
    model = keras.Sequential()
        
    ta_model.hidden_layers(model, params, 0)
    
    model.add(keras.layers.Dense(2, activation=params['last_activation'])) 
    
    model.compile(
        optimizer=params['optimizer'](lr=ta_model.normalizers.lr_normalizer(params['lr'],params['optimizer'])),
        loss=params['losses'],
        metrics=[
            'acc',
            precision, recall, f1score, matthews,
            params['losses'],
        ]
    )

    model_fit = model.fit(
        x=x_train, y=y_train, 
        epochs=params['epochs'], batch_size=params['batch_size'],
        validation_data=(x_val, y_val),
        verbose=0
    )
    
    
    return model_fit, model


Define parameter space to explore for optimization

In [ ]:
ta_params = dict()
ta_params['lr'] = [0.0005, 0.001, 0.0025, 0.005, 0.01, 0.025, 0.05, 0.1, 0.25, 0.5]
ta_params['first_neuron'] = [4, 8, 16, 32]
ta_params['hidden_layers'] = [0, 1, 2, 3]
ta_params['batch_size'] = [2, 4, 16, 32]
ta_params['epochs'] = [25, 50, 150]
ta_params['dropout'] = (0, 0.5, 6)
ta_params['shapes'] = ['brick', 'triangle']
ta_params['optimizer'] = [keras.optimizers.Adam]
ta_params['losses'] = ['categorical_crossentropy']
ta_params['activation'] = ['relu']
ta_params['last_activation'] = ['softmax']

Execute scan

In [ ]:
if control['run_scan']:
    ta_scan = ta.Scan(
        x=X,
        y=Y,
        val_split=0.2,
        seed=32,
        model=get_model,
        fraction_limit=0.25,
        params=ta_params,
        experiment_name=control['experiment_name'],
        clear_session=True,
        print_params=True
    )
    
    ta.Deploy(ta_scan, control['experiment_name'])

Optional upload of results to S3

In [ ]:
if control['upload_results']:
    print('Starting upload of results to S3...')
    
    boto3.client('s3').upload_file('./'+results_filename, control['s3bucket'], results_filename)
    boto3.client('s3').upload_file('./'+deploy_filename, control['s3bucket'], deploy_filename) 
    

## Optimization results

Optional download of results from S3

In [ ]:
if control['download_results']:
    print('Starting download of results from S3...')
    
    boto3.client('s3').download_file(control['s3bucket'], results_filename, './'+results_filename)
    boto3.client('s3').download_file(control['s3bucket'], deploy_filename, './'+deploy_filename)
    

In [ ]:
ta_restore = ta.Restore('./'+deploy_filename)

In [ ]:
df_results = pd.read_csv('./'+results_filename)

Calculate AUC from the best model

In [ ]:
y_pred = ta_restore.model.predict_proba(X)

In [ ]:
print(classification_report(np.argmax(Y, axis=-1), np.argmax(y_pred, axis=-1), output_dict=False))

In [ ]:
auc = roc_auc_score(Y, y_pred, average='weighted')
print('AUC score for best model: %f' % auc)

Visualize results distribution

In [ ]:
sns.jointplot(x="val_acc", y="val_categorical_crossentropy", data=df_results);

In [ ]:
sns.jointplot(x="categorical_crossentropy", y="val_categorical_crossentropy", data=df_results);

Add metric to track underfitting/overfitting

In [ ]:
df_results['loss_diff'] = df_results['val_categorical_crossentropy'] - df_results['categorical_crossentropy']

In [ ]:
diff_t = 0.01

In [ ]:
sns.jointplot(x="val_acc", y="loss_diff", data=df_results)

Show top results

In [ ]:
param_cols = [
    'hidden_layers','activation','batch_size',
    'first_neuron','lr','shapes','dropout','epochs'
]

In [ ]:
df_results[df_results['loss_diff']<diff_t][param_cols+['acc','val_acc','matthews','val_matthews','loss_diff']].sort_values(by=['val_acc'], ascending=False).head(10)


In [ ]:
df_results_melt = df_results[df_results['loss_diff']<diff_t].sort_values(by=['val_acc'], ascending=False).head(10)
df_results_melt['index1'] = df_results_melt.index
df_results_melt = pd.melt(df_results_melt, id_vars=['index1'], value_vars=['categorical_crossentropy', 'val_categorical_crossentropy'])

In [ ]:
plt.figure(figsize=(10,6))
ax = sns.barplot(x="index1", y="value", hue="variable", data=df_results_melt)
plt.legend(loc='lower center')
plt.show()

Create correlation matrix and show values against val_acc

In [ ]:
# encode categorical columns
df_results_cat = df_results[df_results['loss_diff']<diff_t]
df_results_cat["shapes_cat"] = df_results["shapes"].astype('category').cat.codes

In [ ]:
# Compute the correlation matrix
corr = df_results_cat[param_cols+['shapes_cat']+['val_acc','val_matthews','loss_diff']].corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(150, 275, s=80, l=55, n=9, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

In [ ]:
corr['val_acc']

Plot performance over epochs and batch sizes

In [ ]:
sns.catplot(x="epochs", y="val_acc", hue="batch_size", kind="bar", data=df_results_cat);

In [ ]:
sns.catplot(x="epochs", y="loss_diff", hue="batch_size", kind="box", data=df_results_cat);